<a href="https://colab.research.google.com/github/Rintaro0804/ai-practice/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50 # 今回の主役！

# 1. データ準備（いつも通り）
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 前処理：ResNet用に独自の処理が必要です
# ResNetは元々「画素値の平均を引く」などの特殊な処理で学習されているため、それに合わせます
X_train = keras.applications.resnet50.preprocess_input(X_train.astype("float32"))
X_test = keras.applications.resnet50.preprocess_input(X_test.astype("float32"))

# 2. 学習済みモデル（天才）を呼び出す
base_model = ResNet50(
    weights='imagenet',       # ImageNetで学習した「賢い重み」を使う
    include_top=False,        # 最後の「分類層（1000種用）」は捨てる（自分用に作り直すため）
    input_shape=(224, 224, 3) # ResNetが得意な画像サイズ（後で拡大します）
)

# 【重要】天才の知識を上書きしないように「凍結」する
base_model.trainable = False

# 3. モデル構築（天才の上に、自分用の回路をくっつける）
model = keras.Sequential([
    # 入力層: CIFAR-10は小さい(32px)ので、ResNetが見やすい大きさ(224px)に拡大する
    # ※注意: これにより計算量がかなり増えます
    layers.Input(shape=(32, 32, 3)),
    layers.UpSampling2D(size=(7, 7)), # 32x7 = 224ピクセルに拡大

    base_model, # ここにResNet50がドーンと入る

    layers.GlobalAveragePooling2D(), # 情報をギュッと凝縮する
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax') # CIFAR-10用の出力
])

# 4. コンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary() # 構成を見てみよう（パラメータ数に注目！）

# 5. 学習開始（天才なので5エポックで十分なはずです）
print("転移学習スタート...")
history = model.fit(X_train, y_train, epochs=5,
                    batch_size=64, # メモリ不足になる場合は32に減らしてください
                    validation_data=(X_test, y_test))

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ up_sampling2d (UpSampling2D)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,826 (91.99 MB)

 Trainable params: 527,114 (2.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

転移学習スタート...
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 164s 188ms/step - accuracy: 0.6184 - loss: 1.1314 - val_accuracy: 0.7775 - val_loss: 0.6294
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 179ms/step - accuracy: 0.7567 - loss: 0.7007 - val_accuracy: 0.7983 - val_loss: 0.5761
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 179ms/step - accuracy: 0.7779 - loss: 0.6398 - val_accuracy: 0.8095 - val_loss: 0.5453
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 179ms/step - accuracy: 0.7918 - loss: 0.6008 - val_accuracy: 0.8106 - val_loss: 0.5375
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 140s 179ms/step - accuracy: 0.8043 - loss: 0.5611 - val_accuracy: 0.8146 - val_loss: 0.5227


In [2]:
# 1. 封印解除：ResNetの重みを更新できるようにする
base_model.trainable = True

# 【重要】全部を再学習すると時間がかかりすぎる＆崩壊するので
# 「上の方の層（100層以降）」だけを解凍するのがプロの定石です
# ResNet50は約170層あります
fine_tune_at = 140

# 140層目より前は凍結したまま、それ以降だけ動かす
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 2. 再コンパイル（ここが最重要ポイント！）
# 学習率(learning_rate)を、さっきの1/10〜1/100くらい（1e-5）に下げます。
# 急激に書き換えると「記憶喪失」になるからです。
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # 0.00001
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary() # "Trainable params" が増えたことを確認！

# 3. 本気の学習スタート（追加で10エポック）
print("ファインチューニング開始...（ここから精度が伸びます）")
history_fine = model.fit(X_train, y_train,
                         epochs=10, # 追加学習
                         batch_size=64,
                         validation_data=(X_test, y_test))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ up_sampling2d (UpSampling2D)    │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,114,826 (91.99 MB)

 Trainable params: 15,505,162 (59.15 MB)

 Non-trainable params: 8,609,664 (32.84 MB)

ファインチューニング開始...（ここから精度が伸びます）
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 229s 265ms/step - accuracy: 0.7660 - loss: 0.7040 - val_accuracy: 0.8581 - val_loss: 0.4189
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 248ms/step - accuracy: 0.8782 - loss: 0.3491 - val_accuracy: 0.8744 - val_loss: 0.3770
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 248ms/step - accuracy: 0.9208 - loss: 0.2330 - val_accuracy: 0.8841 - val_loss: 0.3641
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 193s 247ms/step - accuracy: 0.9463 - loss: 0.1589 - val_accuracy: 0.8905 - val_loss: 0.3630
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 248ms/step - accuracy: 0.9640 - loss: 0.1106 - val_accuracy: 0.8914 - val_loss: 0.3911
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 211s 270ms/step - accuracy: 0.9792 - loss: 0.0672 - val_accuracy: 0.8916 - val_loss: 0.4188
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 248ms/step - accuracy: 0.9865 - loss: 0.0462 - val_accuracy: 0.8913 - val_loss: 0.4385
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 194s 24